# Tensorflow 現成的模型(pre-made Estimators)

### Estimator
Tensorflow Estimator 是一個高階的API，它支援以下階段的執行：
- 訓練(training)
- 評估(evaluation)
- 預測(prediction)
- 匯出至 Tensorflow serving 模組，準備部署。

Estimator 比 Keras 還是有一些優勢如下：

- Estimator 提供一些 Keras 還在發展中的功能，例如分散式處理(Parameter server)的訓練、TFX的整合。
- 提供一些現成的模型(pre-made Estimators)，包括一般機器學習的模型，例如迴歸、深度學習(DNNClassifier)。

TensorFlow Extended (TFX) 架構，串連整個生產流程。
![](https://ithelp.ithome.com.tw/upload/images/20200923/20001976Xm0cDiLfJt.png)

## 程式參考來源：
- https://www.tensorflow.org/tutorials/estimator/premade
- https://www.tensorflow.org/tutorials/estimator/linear
- https://www.tensorflow.org/guide/estimator

## Iris sample with tf.estimator.DNNClassifier

In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.datasets import load_iris

## 讀取 iris 資料集

In [2]:
ds = load_iris()

In [3]:
df = pd.DataFrame(ds.data, columns=ds.feature_names)
X = ds.data
y = ds.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [4]:
# rename
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'] #, 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
df.columns = CSV_COLUMN_NAMES


In [5]:
# 切割為訓練資料及驗證資料
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
X_train

,SepalLength,SepalWidth,PetalLength,PetalWidth
113,5.7,2.5,5.0,2.0
23,5.1,3.3,1.7,0.5
62,6.0,2.2,4.0,1.0
58,6.6,2.9,4.6,1.3
90,5.5,2.6,4.4,1.2
21,5.1,3.7,1.5,0.4
8,4.4,2.9,1.4,0.2
81,5.5,2.4,3.7,1.0
82,5.8,2.7,3.9,1.2
129,7.2,3.0,5.8,1.6


## 建立 Estimator

In [6]:
# 建立 Estimator input function
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # 將 NumPy Array 轉成 Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # 隨機抽樣
    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [7]:
# 建立 Feature columns 提供給 Estimator 使用
my_feature_columns = []
for key in X_train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
my_feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [8]:
# 建立兩層的深度神經網路(DNN with 2 hidden layers with 30 and 10 hidden nodes each)
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\CHIAWE~1\\AppData\\Local\\Temp\\tmp9vvuba2u', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## 模型訓練

In [9]:
classifier.train(
    input_fn=lambda: input_fn(X_train, y_train, training=True),
    steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
IN

## 模型評估

In [10]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(X_test, y_test, training=False))

print('\n準確度: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-27T11:19:21Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\CHIAWE~1\AppData\Local\Temp\tmp9vvuba2u\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.27528s
INFO:tensorflow:Finished evaluation at 2021-01-27-11:19:21
INFO:tensorflow:Saving dict for global step 500

## 預測

In [11]:
# 預測資料
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

# 預測使用的 Input function
def input_fn_pred(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

# 預測
predictions = classifier.predict(
    input_fn=lambda: input_fn_pred(predict_x))

# 顯示預測結果
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\CHIAWE~1\AppData\Local\Temp\tmp9vvuba2u\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (69.2%), expected "Setosa"
Prediction is "Versicolor" (44.9%), expected "Versicolor"
Prediction is "Virginica" (57.1%), expected "Virginica"
